# Bananadev

This notebook covers how to use whisper from banana.dev, build an audio chain and deploy it in combination with llm to summarize a transcript gathered from an audio file.

In [ ]:
import langchain
from langchain.audio_models import AudioBanana
from langchain.cache import SQLiteCache
from langchain.chains import AudioChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

### Setting the cache

In [ ]:
audio_cache = SQLiteCache(database_path=".langchain.db")

langchain.llm_cache = audio_cache

### Defining the audio model

An `AudioBanana` object has the following arguments:

* `model_key` (str): model endpoint to use;
* `banana_api_key`(optional[str]): banana api key:
* `max_chars` (optional[int]): max number of chars to return.

An `AudioChain` object has the following arguments:
* `audio_model` (AudioBase): the audio model to use;
* `output_key` (str): the task to be performed by the model. Not all the audio models support all the tasks. In the case of AudioBanana only `"transcribe"` is a valid output key. The output key will returned along with the text data gathered from the audio file.




In [ ]:
audio_model = AudioBanana(model_key="[YOUR MODEL KEY]", max_chars=20000)

audio_chain = AudioChain(audio_model=audio_model, output_key="transcribe")

### Setting up a llm and an LLMChain

In [ ]:
llm = OpenAI(temperature=0.7)
template = """Speech: {text}

    Write a short 3 sentence summary of the speech.

    Summary:"""

# note how the input variabòes
prompt_template = PromptTemplate(input_variables=["text"], template=template)

summary_chain = LLMChain(llm=llm, prompt=prompt_template)

### Defining a SimpleSequentialChain and running

In [ ]:
speech_summary_chain = SimpleSequentialChain(
    chains=[audio_chain, summary_chain], verbose=True
)

audio_path = "example_data/ihaveadream.mp3"
speech_summary_chain.run(audio_path)